# 理解嵌入层和线性层之间的区别

- PyTorch中的嵌入层实现了与执行矩阵乘法的线性层相同的功能；我们使用嵌入层的原因是计算效率更高
- 我们将使用PyTorch中的代码示例逐步了解这种关系

In [1]:
import torch

print("PyTorch版本:", torch.__version__)

PyTorch version: 2.3.1


<br>
&nbsp;

## 使用nn.Embedding

In [2]:
# 假设我们有以下3个训练示例，
# 这些可能代表LLM上下文中的token ID
idx = torch.tensor([2, 3, 1])

# 嵌入矩阵的行数可以通过获取最大token ID + 1来确定。
# 如果最高的token ID是3，那么我们需要4行，
# 对应可能的token ID 0, 1, 2, 3
num_idx = max(idx)+1

# 期望的嵌入维度是一个超参数
out_dim = 5

- 让我们实现一个简单的嵌入层：

In [3]:
# 我们使用随机种子来确保可重现性，因为
# 嵌入层中的权重是用小的随机值初始化的
torch.manual_seed(123)

embedding = torch.nn.Embedding(num_idx, out_dim)

我们可以选择性地查看嵌入权重：

In [4]:
embedding.weight

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.3035, -0.5880,  1.5810],
        [ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015],
        [ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096, -0.4076,  0.7953]], requires_grad=True)

- 然后我们可以使用嵌入层来获取ID为1的训练示例的向量表示：

In [5]:
embedding(torch.tensor([1]))

tensor([[ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015]],
       grad_fn=<EmbeddingBackward0>)

- 下面是底层发生的过程的可视化：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/embeddings-and-linear-layers/1.png" width="400px">

- 类似地，我们可以使用嵌入层来获取ID为2的训练示例的向量表示：

In [6]:
embedding(torch.tensor([2]))

tensor([[ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315]],
       grad_fn=<EmbeddingBackward0>)

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/embeddings-and-linear-layers/2.png" width="400px">

- 现在，让我们转换之前定义的所有训练示例：

In [7]:
idx = torch.tensor([2, 3, 1])
embedding(idx)

tensor([[ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096, -0.4076,  0.7953],
        [ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015]],
       grad_fn=<EmbeddingBackward0>)

- 在底层，这仍然是相同的查找概念：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/embeddings-and-linear-layers/3.png" width="450px">

<br>
&nbsp;

## 使用nn.Linear

- 现在，我们将演示上面的嵌入层与在PyTorch中对one-hot编码表示使用`nn.Linear`层实现完全相同的功能
- 首先，让我们将token ID转换为one-hot表示：

In [8]:
onehot = torch.nn.functional.one_hot(idx)
onehot

tensor([[0, 0, 1, 0],
        [0, 0, 0, 1],
        [0, 1, 0, 0]])

- 接下来，我们初始化一个`Linear`层，它执行矩阵乘法 $X W^\top$：

In [9]:
torch.manual_seed(123)
linear = torch.nn.Linear(num_idx, out_dim, bias=False)
linear.weight

Parameter containing:
tensor([[-0.2039,  0.0166, -0.2483,  0.1886],
        [-0.4260,  0.3665, -0.3634, -0.3975],
        [-0.3159,  0.2264, -0.1847,  0.1871],
        [-0.4244, -0.3034, -0.1836, -0.0983],
        [-0.3814,  0.3274, -0.1179,  0.1605]], requires_grad=True)

- 注意PyTorch中的线性层也是用小的随机权重初始化的；为了与上面的`Embedding`层直接比较，我们必须使用相同的小随机权重，这就是为什么我们在这里重新分配它们：

In [10]:
linear.weight = torch.nn.Parameter(embedding.weight.T)

- 现在我们可以在输入的one-hot编码表示上使用线性层：

In [11]:
linear(onehot.float())

tensor([[ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096, -0.4076,  0.7953],
        [ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015]], grad_fn=<MmBackward0>)

正如我们所见，这与我们使用嵌入层时得到的结果完全相同：

In [12]:
embedding(idx)

tensor([[ 0.6957, -1.8061, -1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096, -0.4076,  0.7953],
        [ 1.3010,  1.2753, -0.2010, -0.1606, -0.4015]],
       grad_fn=<EmbeddingBackward0>)

- 底层发生的是第一个训练示例的token ID的以下计算：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/embeddings-and-linear-layers/4.png" width="450px">

- 对于第二个训练示例的token ID：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/embeddings-and-linear-layers/5.png" width="450px">

- 由于每个one-hot编码行中除了一个索引外所有索引都是0（设计如此），这个矩阵乘法本质上与one-hot元素的查找相同
- 在one-hot编码上使用矩阵乘法等同于嵌入层查找，但如果我们处理大的嵌入矩阵时可能效率低下，因为存在大量与零的无效乘法运算